In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

In [2]:
XGB = joblib.load('XGB_Final.pkl')
SVM = joblib.load('SVM_Final.pkl')
LR = joblib.load('LR_Final.pkl')
NB = joblib.load('NB_Final.pkl')
DT = joblib.load('DT_Final.pkl')
KNN = joblib.load('KNN_Final.pkl')

In [3]:
test_data = pd.read_csv('SHP2_test_descriptors_150.csv')
test_data

,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50(microM),TARGET
0,O=C(C(C=C1O)=CC2=CC=C(O)C(O)=C2C1=O)O,11.762075,-1.324292,11.762075,0.104630,0.559777,248.190,240.126,248.032088,92,...,0,0,0,0,0,0,0,0,0.097,1
1,O=C(C(C=C1O)=CC2=CC=C(O)C(O)=C2C1=O)OC,11.891220,-0.890229,11.891220,0.044269,0.524898,262.217,252.137,262.047738,98,...,0,0,0,0,0,0,0,0,0.161,1
2,O=C(C(C=C1O)=CC2=CC=C(O)C(O)=C2C1=O)OCC,11.960761,-0.885233,11.960761,0.026352,0.566824,276.244,264.148,276.063388,104,...,0,0,0,0,0,0,0,0,0.985,1
3,O=C(C(C=C1O)=CC2=CC=C(O)C(O)=C2C1=O)OC(C)C,12.025094,-0.887040,12.025094,0.029269,0.576007,290.271,276.159,290.079038,110,...,0,0,0,0,0,0,0,0,4.87,0
4,O=C(C(C=C1O)=CC2=CC=C(O)C(O)=C2C1=O)N,11.802891,-0.897315,11.802891,0.078991,0.543863,247.206,238.134,247.048072,92,...,0,0,0,0,0,0,0,0,4.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,O=C(NC1=CC=CC=C1[N+]([O-])=O)CCN(C(C)=NC2=C3C4...,12.993000,-0.542407,12.993000,0.020148,0.524557,398.444,380.300,398.104876,144,...,0,0,0,0,0,1,0,0,34.826,0
146,O=C(C1=CC2=C(N(C)C(C3=CC=CC=C3)=C2C4=CN(CCC(NC...,12.357815,-1.223568,12.357815,0.143555,0.313800,481.512,458.328,481.175004,180,...,0,0,0,0,0,0,0,0,86,0
147,O=C(C1=CC2=C(N(C)C(C3=CC=CC=C3)=C2C4=CN(CC(NC5...,12.847668,-1.238342,12.847668,0.069408,0.233485,543.583,518.383,543.190654,202,...,0,0,0,0,0,0,0,0,9.3,0
148,O=C(C1=CC2=C(N(C)C(C3=CC=CC=C3)=C2C4=CN(CCC(NC...,12.712730,-1.227003,12.712730,0.156276,0.204736,557.610,530.394,557.206304,208,...,0,0,0,0,0,0,0,0,5.5,0


In [4]:
final_features = ['MaxPartialCharge', 'FpDensityMorgan2', 'BCUT2D_CHGLO', 'BCUT2D_MRHI',
       'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA3', 'SlogP_VSA3', 'SlogP_VSA8',
       'EState_VSA6', 'NumHAcceptors', 'NumSaturatedCarbocycles',
       'fr_bicyclic', 'TARGET']

In [5]:
test_data = test_data[final_features]
test_data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET
0,0.335343,1.666667,-1.988112,5.961393,5.42879,6.066367,0.000000,0.000000,10.772448,0.000000,5,0,1,1
1,0.337506,1.789474,-1.993067,5.968264,5.42879,6.066367,0.000000,4.736863,10.772448,0.000000,6,0,1,1
2,0.337772,1.850000,-2.002685,5.968802,5.42879,6.066367,0.000000,4.736863,10.772448,6.923737,6,0,1,1
3,0.338038,1.761905,-2.062196,5.969423,5.42879,6.066367,0.000000,4.736863,10.772448,13.847474,6,0,1,0
4,0.248358,1.722222,-1.988933,5.984624,11.33597,6.066367,0.000000,0.000000,10.772448,0.000000,5,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.292242,2.142857,-2.102211,7.184306,5.90718,12.132734,9.551078,24.180937,10.216621,24.326890,7,0,3,0
146,0.338995,1.694444,-2.066425,6.071176,5.90718,53.744322,19.560969,18.386966,33.417684,10.878647,7,0,1,0
147,0.338995,1.487805,-2.071661,6.071856,5.90718,78.009790,19.560969,18.386966,44.544587,6.196844,7,0,1,0
148,0.338995,1.500000,-2.071839,6.071461,5.90718,78.009790,19.560969,18.386966,44.544587,10.878647,7,0,1,0


In [6]:
X_test = test_data.iloc[:,:-1].values
y_test = test_data.iloc[:,-1].values

In [7]:
def plot_roc_curve(X_test, y_test, model, model_name):
    y_pred = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    auc = round(metrics.roc_auc_score(y_test, y_pred), 4)
    plot_label = model_name + " AUC= " + str(auc)
    return plt.plot(fpr,tpr,label=plot_label)

In [8]:
plt.figure(figsize=(16,10), dpi=1200)

# XGB
y_pred1 = XGB.predict_proba(X_test)[:, 1]
fpr1, tpr1, th1 = metrics.roc_curve(y_test, y_pred1)
auc1 = round(metrics.roc_auc_score(y_test, y_pred1), 4)
plt.plot(fpr1,tpr1,label="XGBOOST, AUC="+str(auc1))

# # # SVM
# y_pred2 = SVM.predict_proba(X_test)[:, 1]
# fpr2, tpr2, _ = metrics.roc_curve(y_test, y_pred2)
# auc2 = round(metrics.roc_auc_score(y_test, y_pred2), 4)
# plt.plot(fpr2,tpr2,label="SVM, AUC="+str(auc2))

# # # LR
# y_pred3 = LR.predict_proba(X_test)[:, 1]
# fpr3, tpr3, _ = metrics.roc_curve(y_test, y_pred3)
# auc3 = round(metrics.roc_auc_score(y_test, y_pred3), 4)
# plt.plot(fpr3,tpr3,label="LR, AUC="+str(auc3))

# # # NB
# y_pred4 = NB.predict_proba(X_test)[:, 1]
# fpr4, tpr4, _ = metrics.roc_curve(y_test, y_pred4)
# auc4 = round(metrics.roc_auc_score(y_test, y_pred4), 4)
# plt.plot(fpr4,tpr4,label="NB, AUC="+str(auc4))

# DT
y_pred5 = DT.predict_proba(X_test)[:, 1]
fpr5, tpr5, th2 = metrics.roc_curve(y_test, y_pred5)
auc5 = round(metrics.roc_auc_score(y_test, y_pred5), 4)
plt.plot(fpr5,tpr5,label="DT, AUC="+str(auc5))

# KNN
y_pred6 = KNN.predict_proba(X_test)[:, 1]
fpr6, tpr6, th3 = metrics.roc_curve(y_test, y_pred6)
auc6 = round(metrics.roc_auc_score(y_test, y_pred6), 4)
plt.plot(fpr6,tpr6,label="KNN, AUC="+str(auc6))

plt.rcParams.update({'font.size': 18})
plt.title('ROC-AUC Curve')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#add legend
plt.legend()
plt.savefig('..\images\ROC_AUC_Curve.png')

E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
